https://www.data.go.kr/data/15097922/fileData.do

In [1]:
import requests
from multiprocessing import Pool
#BeautifulSoup
from bs4 import BeautifulSoup as bs 
#Selenium
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys


import time
from tqdm import tqdm #showing process bar
import pandas as pd
import csv
import openpyxl #read & write excel file in Python
import re
import gc
import numpy as np
import random

In [2]:
pg = '1'
pp = '50'
url = f'http://www.cgs.or.kr/business/esg_tab04.jsp?pg={pg}&pp={pp}&skey=&svalue=&sfyear=2021&styear=2021&sgtype=TOTAL&sgrade=#ui_contents'

url

SyntaxError: invalid syntax (Temp/ipykernel_26788/709873281.py, line 4)

In [ ]:
#chromdirver path 지정
driver = webdriver.Chrome('C:/Users/cathy/Jupyter/chromdriver')

#url 접속
driver.get(url)

In [ ]:
#get 요청하고, response 받음
req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#403오류 해결

company = []
code = []
esg_score = []
e_score = []
s_score = []
g_score = []
year = []


if (req.status_code == 200):
    html = req.text
    soup = bs(html, 'html.parser')
    selector = ''
    elements = soup.select(selector)
    
    for e  in elements:
        esg_list.append(e)
        
        
    
else:
    print(f'오류: {response.status_code}')